In [ ]:
import numpy as np
import os
os.environ['TF_ENABLE_ONEDNN_OPTS']='0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import pandas as pd
import tensorflow as tf 
from tensorflow import keras
import matplotlib.pyplot as plt

In [ ]:
ALL_DATA_DIR = '/media/windows/'
training = ALL_DATA_DIR + 'training/'
validation = ALL_DATA_DIR + 'validation/'
scaled = ALL_DATA_DIR + 'scaled/'
scaled_wo_rotate = ALL_DATA_DIR + "scaled_wo_rotate/"

scaled_wo_rotate_training = scaled_wo_rotate + "training/"
scaled_wo_rotate_validation = scaled_wo_rotate + "validation/"

datFile = training + "shape_predictor_68_face_landmarks.dat"

In [ ]:
batch_size = 32
img_height = 224
img_width = 224

In [ ]:
validation_s = tf.keras.utils.image_dataset_from_directory(scaled_wo_rotate_validation, labels = 'inferred',
            image_size=(img_height,img_width), batch_size=batch_size, seed=42).prefetch(1)

In [ ]:
training_s = tf.keras.utils.image_dataset_from_directory(scaled_wo_rotate_training, labels = 'inferred',
            image_size=(img_height,img_width), batch_size=batch_size, seed=42).prefetch(1)

In [ ]:
base_model = keras.applications.EfficientNetB4(input_shape=(224,224,3), include_top = False, weights='imagenet')

In [ ]:
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(7, activation='softmax')(avg)
model = keras.Model(inputs=base_model.input, outputs=output)

In [ ]:
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
model.compile(optimizer="Adam",
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(training_s, validation_data=validation_s, epochs=5)

In [ ]:
model.save('0920_EffiB4_frozenfit.h5')

In [ ]:
for layer in base_model.layers:
    layer.trainable = True

In [ ]:
opt = keras.optimizers.Adam(learning_rate=1e-5)

In [ ]:
es = keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
history = model.fit(training_s, validation_data=validation_s, epochs=30, callbacks=[es])

In [ ]:
model.save('0920_EffiB4_new.h5')